# Capturando tuits

[Pablo Haya](http://pablohaya.com), Ultima versión: Mar-2018

## Objetivo

Este _notebook_ presenta como utiliza Python para obtener tuits a través de la API pública de Twitter.

### Obtención de credenciales de acceso


La captura de tuits se va realizar empleando la API pública que provee Twitter. Antes de empezar es preciso dar alta una aplicación en Twitter, y obtener las credenciasles de acceso que autoricen a la aplicación a descargarse menciones desde la API.

Los pasos para dar de alta una aplicación son:

* Crea una cuenta en [Twitter](https://twitter.com/) si no se dispusiera antes.
* Ve a https://apps.twitter.com/ y conectate con tu usuario.
* Haz click en _Create New App_.
* Rellena el formulario, incluido la dirección de website, aunque no se disponga.
* Acepta los términos y condiciones y crearla.
* En la página siguiente, haz click en _Keys and Access Tokens_ y copia tu _API key_ y _API secret_.
* Baja al final de la página, haz clic en _Create my access token_, y copia _Access token_ y _Access token secret_ en los huecos correspondientes.

### Configuración de la conexión a la API

Una vez creada la aplicación pasamos al script propiamente dicho que primeramente requiere importar la bibliotecas necesarias. Para poder acceder a la API de Twitter se va a utilizar la biblioteca [tweepy](http://www.tweepy.org/). También son necesarias la biblioteca `json`, para manipular los tweets en este formato, y la biblioteca `time` como veremos más adelante.

In [ ]:
# Bibliotecas necesarias
import json
import time
import tweepy
from tweepy.streaming import StreamListener

Es preciso añadir las credenciales obtenidas al crear la aplicación sustituyéndolas en el siguiente extracto de código: 

In [ ]:
# Credenciales de acceso a la API de Twitter
access_token = "INSERTA ACCESS TOKEN"
access_token_secret = "INSERTA ACCESS TOKEN SECRET"
consumer_key = "INSERTA CONSUMER KEY"
consumer_secret = "INSERTA CONSUMER SECRET"

Se realiza la autenticación empleando las credeciales definidas anteriormente, y se inicializa la conexión.

In [ ]:
# Autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

### Extraer tuits de la propia cronología

La opción más directa para obtener un conjunto de tuits es acceder a la cronología (_timeline_) del propio usuario que ha creado la aplicación. Por defecto, la función [home_timeline](http://docs.tweepy.org/en/v3.5.0/api.html#API.home_timeline) permite acceder a los últimos 20 tuits publicados en la cronología.

La descarga no es inmediata de manera que es posible que tarde unos segundos en aparecer el resultado. Mientras que se estén capturando aparecerá un asterisco [*] en vez del identificador de la celda. 

In [ ]:
api = tweepy.API(auth)

# Función que transforma un objeto Status en json
# No está documentado en la API
def to_json(status):    
    json_str = json.dumps(status._json)
    return json_str

# Función que escribe en un fichero un conjuto de tuits en formato json
# Los tuits se añaden al fichero, y en caso de que no existe se crea
def write_json(tweets, filename):
    n = 0
    try:
        with open(filename, 'a') as outfile:
            for tweet in tweets:
                outfile.write(to_json(tweet))
                n = n + 1
    except (IOError, OSError, Failure) as e:
        pass
    finally:
        return n

In [ ]:
public_tweets = api.home_timeline()
n = write_json(public_tweets, 'home_timeline.json')
print("Guardados %d tuits" % n)

# Imprime únicamente el texto cada tuit a modo de comprobación
for tweet in public_tweets:
    print(tweet.text)

### Acceder a los tuits publicados por un  usuario

Es igual de fácil acceder a los tuits publicados por usuario mediante la función [user_timeline](http://docs.tweepy.org/en/v3.5.0/api.html#API.user_timeline). En este caso nos vamos a descargar los tuits [KDnuggets (@kdnuggets)](https://twitter.com/kdnuggets). El nombre de usuario se añade como parámetro de la función sin la @. El [máximo número de tuits](# https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline.html) que se pueden descargar en cada llamada es de 3.200

In [ ]:
# Nos descargamos los 150 últimos tuits del timeline
public_tweets = api.user_timeline("KDnuggets", count=150)
n = write_json(public_tweets, 'KDnuggets_timeline.json')
print("Guardados %d tuits" % n)

# Imprimos únicamente los 20 primeros tuits
for tweet in public_tweets[1:20]:
    print(tweet.text)

### Obteniendo cualquier tuit 

En general, podemos preguntar a la API para poder descargarnos cualquier tuit que contenga los términos queramos medianta la función [search](http://docs.tweepy.org/en/v3.5.0/api.html#API.search). Es importante tener en cuenta que la API pública tiene ciertas restricciones que impide descargarse todos los posibles tuits cuando el volumen es muy elevado.

Para ello se define un consulta que se codifica como un array de términos que se interpreta como un OR. Así para descargar tuits que contenga el término "hadoop" o "spark", se codificaría como `["hadoop", "spark"]` 

También se pueden incluir términos relacionados mediante un AND si se separan con un espacio, de manera que es preciso que estén ambos términos para que se capture el tuit. En este caso, sería necesario especificar mejor que tuits habría capturar relativos a _Spark_, ya que es un palabra que puede aparecer en otros contextos distintos de tecnologías de Big Data.

Por ejemplo: si quisiéramos la siguiente consulta "hadoop" OR ("apache" AND "spark") se codificará como `["hadoop", "apache spark"]`.

In [ ]:
public_tweets = api.search(["hadoop", "apache spark"])
n = write_json(public_tweets, 'hadoop_or_apachespark.json')
print("Guardados %d tuits" % n)

for tweet in public_tweets:
    print(tweet.text)

### Capturando tuits en tiempo real

A veces queremos capturar tuit de manera continuada. Para ello podemos definir una consulta que se descarge un flujo de tuits a medida que se van publicando en Twitter.

Primeramente hay que declararse una clase de tipo `StreamListener` que gestione el flujo de tuits defiendo que se quiere hacer con cada uno, y cuando se termina la captura. 

La siguiente clase se encarga de guardar en fichero cada tuit recibido. Se inicializa con el nombre del fichero, y con el tiempo durante el cual se estarán capturando los tuits. Cada vez que se recibe un tuit se llama el método `on_data` que se encarga de comprobar si se ha excedido la duración de la captura, y de guardar el tuit en el fichero. El retorno de este función indica si se continua `True` o no `False` con la descarga de tweets.

In [ ]:
class FileStreamListener(StreamListener):
    # requiere el nombre del fichero donde almacenar los tweets, 
    # y el tiempo límite en segundos con el que se configura la captura.
    def __init__(self, filename, time_limit=30):
        self.start_time = time.time()
        self.limit = time_limit
        self.save_file = open(filename, 'a')
        self.n = 0 # numero de tuits procesados
        super(FileStreamListener, self).__init__()

    def on_data(self, tweet):
        if (time.time() - self.start_time) < self.limit:
            #print(tweet)
            self.save_file.write(tweet)
            self.save_file.write('\n')
            self.n = self.n + 1
            return True
        else:
            self.save_file.close()
            return False

A continuación se inicializa el flujo de tuits, y se configura con una consulta similar a la hicimos anteriormente para extraer tuits sobre "Hadoop" y "Apache Spark".

Recordar que es preciso esperar hasta que venza el tiempo de la captura definido (1 min.) para disponer del archivo con los tweets. Mientras que se estén capturando aparecerá un asterisco [*] en vez del identificador de la celda.  

In [ ]:
# Conexión a stream data
listener = FileStreamListener(filename='twitter_stream_data.json', time_limit=60)
stream = tweepy.Stream(auth, listener)
# Captura del stream de Twitter actualizaciones de estado que contenga los términos
stream.filter(track = ['hadoop', 'apache spark'])

In [ ]:
print("Guardados %d tuits" % listener.n)

### Capturando tendencias en Twitter

Para terminar vamos a ver como se puede obtener aquellos términos que están siendo tendencia (_trending topics_) en un momento dado. Hay que tener en cuenta que las tendencias se definen por zonas geográficas de manera que es precio especificar primeramente la región de interés sobre la cual se quieren obtener las tendencias. Cada región se identifica con un _woeid_. Por ejemplo, los _trending topics_ mundiales son el _woeid_ 1.

A continuación se consultan todas las posibles regiones que existen:

In [ ]:
locations = api.trends_available()
locations

Nos fijamos en una localización en concreto (por ejemplo, Madrid)

In [ ]:
for loc in locations:
    if loc['name'] == 'Madrid':
        mad_woeid = loc['woeid']
        break
mad_woeid

Obtenemos todas las tendencias de Madrid

In [ ]:
mad_tt = api.trends_place(mad_woeid)
mad_tt

Recorremos las tendencias, y nos descargamos los últimos tuits para cada una de ellas a partir del atributo ´query´, y los guardamos en fichero.

In [ ]:
# obtenemos todas las queries
queries_tt = list(map(lambda tweet: tweet['query'], mad_tt[0]['trends']))

for q in queries_tt:
    public_tweets = api.search(q)
    write_json(public_tweets, 'trending_topics.json')

# Imprimimos unicamente los tuits de la última query
for tweet in public_tweets:
    print(tweet.text)

### Ejercicios

1. Modificar la descarga de tuits en tiempo real de manera incluyan los términos "logistic regression" o "deep learning".
* Crear un nuevo `StreamListener` que termine de capturar tuits cuando haya descargado un número predeterminado. 
* Recuperar todos los _trending topics_ de España, Madrid y Barcelona en un único archivo.
